# Making computation Fast

* A review of what we've covered thus far
* an introduction to "fast" single tasking programs
* an introduction to multi tasking programs
* a synthesis of single and multi tasking programs

## Matrices

We've worked with a few algorithms a lot.  Now we'll introduce a new data structure - the matrix.  A matrix mathematically speaking is a notational convention.  More or less a matrix is just a vector of vectors.  And a vector is just the coefficients of a linear equation.  We've been implicitly using mathematical vectors since we introduced equations in lecture three.  To remind you, let's look at some python code, representing a vector:

In [ ]:
from functools import partial
import itertools


def arange(start, stop, step):
    iterator = start
    while iterator < stop:
        yield iterator
        iterator += step

        
def equation(coefficients, constant, variables):
    return sum([coefficients[index]*variables[index] 
                for index in range(len(variables))]) + constant


def find_zero_eq(coefficients, constant, episolon):
    eq = partial(equation, coefficients, constant)
    value_range = list(arange(-10, 10, 0.1))
    values = list(itertools.permutations(value_range, len(coefficients)))
    for value in values:        
        if abs(eq(value)) < episolon:
                return value

print(find_zero_eq([1], 7, 0.1))
print(find_zero_eq([1, 2], 7, 0.1))
print(find_zero_eq([1, 2, 3], 7, 0.1))
print(find_zero_eq([1, 2, 3, 4], 7, 0.1))

The above code should appear familiar because it's the same code we saw in lecture 3.  Notice the list we pass in to `find_zero_eq`.  These different lists are all mathematical vectors, because they represent the coeficients of an equation.  A matrix is a similar idea, except instead of having a system depend on only one equation, it depends on multiple.  Let's see a simple example of this before we formalize the notation:

## A first example of matrices, Supply and Demand

Let's say that we own and operate a business that sells candy bars.  Assume there is implicit demand for candy bars and many other candy bar sellers.  However, assume our costs are perhaps slightly different from other candy bar sellers, so we can choose what price to sell our candy bar at, subject to our ability to supply candy.  We can also assume we have some supply control because most people aren't patient about candy, they want it when they want it.  So if someone happens by your shop, the price may actually effect their decision to buy the candy or not.

So let's say you have a supply function like:

`price = 2*quantity + 3`

What this equation says is as a business owner, as the price increases you'll want to supply more candy bars, because the perception is, you'll make more money.  

Let's also say there is a demand function like:

`price = 3 - 2*quantity`

This equation says the price you can charge is subject to how many candy bars you make.  So if your shop is filled to the brim and your the only local vendor around, folks will likely be able to always satisfy their craving for candy.  And if you only make a single candy bar, folks will be willing to pay a lot to be the only one to get it.  Because candy is delicious.

Given the above equations we need both the supply and demand equation to find the unique equilibrium of price and quantity such that the market for candy bars is in balance.  This balancing point, all other things held constant, will be the price that maximizes utility for the buyers of candy bars as well as the sellers of candy bars.  And since we are treating our example as idealized, we are in a capitalism regime where the intention is to maximize utility for all.  However, even with the introduction of immorality and therefore businesses or agents acting in subversive ways, all economies are still subject to the above laws generally speaking and therefore understanding this idealized situation still yields value.

So!  Now that we have our supply and demand equations, how do we find the unique price, quantity that yields our equilibrium?  With matrices of course!

First we'll need to turn these equations into the appropriate form:

Supply:

`price - 2*quantity = 3`

Demand:

`price + 2*quantity = 3`

We need to do this so all the variables are on one side of the equation.  Now we can do this:

Supply/Demand Matrix:

```
[[1 -2][[3]
[1  2]][3]]
```

Now we have the two resulting mathematical objects, a matrix describing the coeficients of the two equations and a vector describing the solution space.  We'll do something called row reduction on the matrix which will produce unique solutions for the equations.  Before we carry out these operations notice that if our matrix looks like this:

```
[[1 0][[a
[0  1]][b]]
```

Then whatever our variables are, they are equal to a and b respectively.  Because the matrix:

```
[[1 0]
[0  1]]
```

Translates to

```
x 
y
```

With the `[a b]` vector this becomes:

```
x = a
y = b
```

Which solves x and y uniquely!  So now we'll do row reduction to create a matrix which looks like the above one.  And then we'll apply the changes we used on the matrix, onto the solution vector as well.  Which will in turn solve our system of equations!

```
[[1 -2]  R[1] = R[1] + R[2]  [[2  0]    R[1] = R[1]/2   [[1  0]   R[2] = R[2] - R[1]  [[1  0]  R[2] = R[2]/2
[1  2]]  ----------------->  [1   2]]   ------------->  [1   2]]  ----------------->   [0  2]] ------------>

[[1  0]
 [0  1]]
 ```
 
Now we'll simply apply the changes to the matrix to the vector:

```
[[3]  R[1] = R[1] + R[2]  [[6]   R[1] = R[1]/2   [[3]   R[2] = R[2] - R[1] [[3]  R[2] = R[2]/2  [[3]
[3]]  ----------------->   [3]]  ------------->   [3]]  -----------------> [0]]  ------------>   [0]]

So the unique solution is:

price = 3
quantity = 0
```

What this ends up telling us is it's not possible to optimally operate our candy bar shop.  Which is pretty sad.  So this means we shouldn't open our candy bar shop.  Or we need to change our supply equation by becoming more efficient at producing candy bars.  Of course, it could also be the case that this is truly optimal because candy is bad for you :P.  But that's not really an economic analysis.

In any event, we are now ready to write a program which does row reduction for us, by solving a bunch of very simple equations.  Notice, what we really want is to make the numbers in the matrix that are not on the diagonal zero.  And then we want to make the numbers that are on the diagonal 1.  And then we need to store those operations and apply them to solution matrix.


In [1]:
from functools import partial
from random import random
from sys import setrecursionlimit

def eq_solver(val, func):
    if func(val) > 0:
        val -= 1
        return eq_solver(val, func)
    elif func(val) < 0:
        val += 1
        return eq_solver(val, func)
    else:
        return round(val, 5)
    
    
def eq_float_solver(val, eq, epsilon=0.0001, step_size=0.001, debug=False):
    if abs(eq(val)) < epsilon:
        return round(val, 5)
    elif eq(val) > 0:
        if debug:
            val -= step_size
            return eq_float_solver(val, eq)
        else:
            val -= step_size
            return eq_float_solver(val, eq)
    elif eq(val) < 0:
        if debug:
            val += step_size
            return eq_float_solver(val, eq)
        else:
            val += step_size
            return eq_float_solver(val, eq)
    else:
        return round(val, 5)


def eq_diag_solver(val, eq, epsilon=0.0001, step_size=0.001):
    if abs(eq(val)) - 1 < epsilon:
        print("got here with val", val)
        return round(val, 5)
    elif eq(val) > 1:
        val -= step_size
        if val == 0:
            val -= step_size
        return eq_float_solver(val, eq)
    elif eq(val) < 1:
        val += step_size
        if val == 0:
            val += step_size
        return eq_float_solver(val, eq)
    else:
        return round(val, 5)
    

def arange(start, stop, step):
    cur = start
    while start < stop:
        yield cur
        cur += step

        
def iterative_solver(eq, start, stop, epsilon=0.0001, step_size=0.001):
    for val in arange(start, stop, step_size):
        if abs(eq(val)) < epsilon:
            return round(val, 5)


def flatten(matrix):
    listing = []
    for row in matrix:
        for elem in row:
            listing.append(elem)
    return listing


def solve_matrix(matrix):
    flattened_matrix = flatten(matrix)
    largest_value = max(flattened_matrix)
    num_zeros = len(str(largest_value))
    val_range = int("1" + "0"*num_zeros)
    steps = []
    cur_matrix = matrix
    for index in range(len(matrix)):
        col_index = index
        row_index = index
        cur_matrix, step = solve_diag(cur_matrix[col_index][row_index],
                                      row_index, col_index, cur_matrix, val_range)
        steps.append(step)
    for row_index in range(len(matrix[0])):
        for col_index in range(len(matrix)):
            if col_index == row_index:
                continue
            print("got here")
            print(cur_matrix)
            step = solve_vector(cur_matrix[col_index][row_index], 
                                row_index, col_index, cur_matrix, val_range)
            print("got here too")
            cur_matrix = linear_combination(cur_matrix, step)
            print(col_index, row_index)
            steps.append(step)
    return steps, cur_matrix


def linear_combination(matrix, step):
    update_row = step[0]
    other_row = step[1]
    transformer = step[2]
    for index in range(len(matrix[0])):
        matrix[update_row][index] = transformer(
            matrix[update_row][index], matrix[other_row][index])
    return matrix
    

def solve_diag(elem, cur_elem_idx, diag_index, matrix, magnitude):
    reciprical = 1/elem
    matrix[diag_index] = [elem*reciprical 
                          for elem in matrix[diag_index]]
    operation = lambda coef, elem: elem*coef
    op = partial(operation, reciprical)
    step = [diag_index, cur_elem_idx, op, None, reciprical, "rescale"]
    return matrix, step


def solve_vector(elem, cur_elem_idx, diag_index, matrix, magnitude):
    for row_index, matrix_row in enumerate(matrix):        
        if matrix_row[cur_elem_idx] != 0 and row_index != diag_index:
            other_elem = matrix_row[cur_elem_idx]
            other_row = row_index
            break
    eq_to_solve = lambda elem, other_elem, coef: elem + coef*other_elem
    to_solve = partial(eq_to_solve, elem, other_elem)
    start, stop = magnitude*-1, magnitude
    coef = iterative_solver(to_solve, start=start, stop=stop)
    operation = lambda coef, elem, other: elem + coef*other
    op = partial(operation, coef)
    return [diag_index, cur_elem_idx, op, other_row, coef, "linear_combo"]


def apply_steps(vector, steps):
    for step in steps:
        if step[-1] == "linear_combo":
            idx = step[0]
            other_idx = step[3]
            coef = step[4]
            vector[idx] = vector[idx] + coef*vector[other_idx]
        else:
            idx = step[0]
            coef = step[4]
            vector[idx] = vector[idx]*coef
    return vector


#setrecursionlimit(10000)
if __name__ == '__main__':
    matrix = [[1, -2, 3], [1, 2, 1], [4, 2, 3]]
    steps, cur_matrix = solve_matrix(matrix)
    print(apply_steps([3, 3, 7], steps))


RecursionError: maximum recursion depth exceeded

Now that we've created the ability to solve linear systems.  We can add all our apply steps into a new matrix and simply multiply the two matrices together.  

Before we go through how to combine these transformation steps.  Let's see how to do matrix multiplication:

The general algorithm is the following:

* take row i of the Left Hand Side Matrix
* take column j of the Right Hand Side Matrix
* Multiply element k of the row vector with element k of column vector  
* Then sum all products together
* place the result in row i column j

Let's look at a practical example:

```
[[1 2]     *   [[2 1]     =    [[1*2 + 2*4  1*1 + 2*3]
 [3 4]]         [4 3]]          [3*2 + 4*4  1*3 + 4*3]]

= [[10 7]
   [22 15]]
```

The code is relatively straight forward:

In [6]:
def transpose(matrix):
    new_matrix = [[0 for _ in range(len(matrix[0]))] for _ in range(len(matrix))]
    for row_index in range(len(matrix)):
        for col_index in range(len(matrix[0])):
            new_matrix[col_index][row_index] = matrix[row_index][col_index]
    return new_matrix


def matrix_multiply(A, B):
    new_matrix = [[0 for _ in range(len(A[0]))] for _ in range(len(A))]
    B_transpose = transpose(B)
    for row_A in range(len(A)):
        for col_B in range(len(B_transpose)):
            new_matrix[row_A][col_B] = sum([elem[0]*elem[1] 
                                            for elem in zip(A[row_A], B_transpose[col_B])])
    return new_matrix

matrix_multiply([[1, 2], [3, 4]], [[2, 1], [4, 3]])    

[[10, 7], [22, 15]]

It turns out this method is actually quiet "slow".  We can do better!  But how?  Let's define a matrix class, and write a recursive matrix multiplier:

In [8]:
class Matrix:
    def __init__(self,matrix):
        self.matrix = matrix
            
    def get_size(self):
        return len(self.matrix),len(self.matrix[0])

    def pprint(self):
        for row in self.matrix:
            print(row)
                
        
    def to_array(self):
        return self.matrix
        
    def get_elem(self,row,col):
        return self.matrix[row][col]    
        
    def __add__(self,other):
        row_size,col_size = self.get_size()
        new_matrix = []
        for row in range(row_size):
            new_matrix.append([elem+other.matrix[row][ind] for ind,elem in enumerate(self.matrix[row])])
        return Matrix(new_matrix)
    
    def __sub__(self,other):
        row_size,col_size = self.get_size()
        new_matrix = []
        for row in range(row_size):
            new_matrix.append([elem-other.matrix[row][ind] for ind,elem in enumerate(self.matrix[row])])
        return Matrix(new_matrix)

    def simple_multiplication(self,A,B):
        row_size,col_size = self.get_size()
        new_matrix = [[0 for i in range(row_size)] for j in range(row_size)]
        for i in range(row_size):
            for k in range(row_size):
                for j in range(row_size):
                    new_matrix[i][j] += A[i][k] * B[k][j]
        return Matrix(new_matrix)
    
    def __mul__(self,other):
        row_size,col_size = self.get_size()
        if row_size <= 2:
            return self.simple_multiplication(self.matrix,other.matrix)
        else:
            new_size = row_size//2
            A = [[0 for j in range(new_size)] for i in range(new_size)]
            B = [[0 for j in range(new_size)] for i in range(new_size)]
            C = [[0 for j in range(new_size)] for i in range(new_size)]
            D = [[0 for j in range(new_size)] for i in range(new_size)]

            E = [[0 for j in range(new_size)] for i in range(new_size)]
            F = [[0 for j in range(new_size)] for i in range(new_size)]
            G = [[0 for j in range(new_size)] for i in range(new_size)]
            H = [[0 for j in range(new_size)] for i in range(new_size)]

            for i in range(new_size):
                for j in range(new_size):
                    A[i][j] = self.matrix[i][j]
                    B[i][j] = self.matrix[i][j+new_size]
                    C[i][j] = self.matrix[i + new_size][j]
                    D[i][j] = self.matrix[i + new_size][j + new_size]

                    E[i][j] = other.matrix[i][j]
                    F[i][j] = other.matrix[i][j+new_size]
                    G[i][j] = other.matrix[i + new_size][j]
                    H[i][j] = other.matrix[i + new_size][j + new_size]

            A = Matrix(A)
            B = Matrix(B)
            C = Matrix(C)
            D = Matrix(D)
            E = Matrix(E)
            F = Matrix(F)
            G = Matrix(G)
            H = Matrix(H)
            
            p1 = A*(F-H)
            p2 = (A+B)*H
            p3 = (C+D)*E
            p4 = D*(G-E)
            p5 = (A+D)*(E+H)
            p6 = (B-D)*(G+H)
            p7 = (A -C)*(E+F)

            c11 = p5 + p4 - p2 + p6
            c12 = p1 + p2
            c21 = p3 + p4
            c22 = p1+ p5 - p3 - p7

            final = [[0 for j in range(row_size)] for i in range(row_size)]
            for i in range(new_size):
                for j in range(new_size):
                    final[i][j] = c11.matrix[i][j]
                    final[i][j+new_size] = c12.matrix[i][j]
                    final[i + new_size][j] = c21.matrix[i][j]
                    final[i + new_size][j + new_size] = c22.matrix[i][j]
            return Matrix(final)
            
A = Matrix([[1,2], [3,4]])
B = Matrix([[2, 1], [4, 3]])
(A*B).pprint()

[10, 7]
[22, 15]


In [ ]:
#https://github.com/flennerhag/mlens <- include in parallelized computation
#https://github.com/kubeflow/kubeflow <- include this for deployment